In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_gen = ImageDataGenerator(rotation_range=20, # rotate the image 20 degrees
                               width_shift_range=0.10, # Shift the pic width by a max of 5%
                               height_shift_range=0.10, # Shift the pic height by a max of 5%
                               #rescale=1/255, # Rescale the image by normalzing it.
                               # note: not using rescale because we're using preprocessing function
                               shear_range=0.1, # Shear means cutting away part of the image (max 10%)
                               zoom_range=0.1, # Zoom in by 10% max
                               horizontal_flip=True, # Allo horizontal flipping
                               fill_mode='nearest', # Fill in missing pixels with the nearest filled value
                               #validation_split=0.1, # adding validation split
                               preprocessing_function=tf.keras.applications.inception_v3.preprocess_input
                               # note that inception_v3 requires the above preprocessing function to be used
                               # to set values between -1 & +1
                              )


In [ ]:
train_gen=image_gen.flow_from_directory('drive/My Drive/TRAIN')

Found 5890 images belonging to 37 classes.


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling2D, GlobalAveragePooling2D

base_model = InceptionV3(
    weights='imagenet', include_top=False, 
    input_shape=(400,400,3) #pooling='avg'
)

87916544/87910968 [==============================] - 1s 0us/step


In [ ]:
for layer in base_model.layers:
    layer.trainable = False
# ensuring we don't train the base model
base_model.trainable = False
x = base_model.output
# passing Inception output via pooling layer
x = tf.keras.layers.GlobalAveragePooling2D()(x)
#x = tf.keras.layers.Dropout(0.2)(x)  # preventing overfitting with dropout
# Flattening the inputs to custom layers
#x = Flatten()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
#x = Dense(50, activation='relu')(x)
# and a logistic layer 
predictions = Dense(37, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=[tf.keras.metrics.Precision()])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stop = EarlyStopping(monitor='val_loss',patience=3)

In [ ]:
train_image_gen = image_gen.flow_from_directory('drive/My Drive/TRAIN',
                                               target_size=(400,400),
                                                color_mode='rgb',
                                               batch_size=16,
                                               class_mode='categorical', 
                                                #subset='training'
                                               )

Found 5890 images belonging to 37 classes.


In [ ]:
# # validation images
# test_image_gen = image_gen.flow_from_directory(train_path,
#                                                target_size=image_shape[:2],
#                                                 color_mode='rgb',
#                                                batch_size=batch_size,
#                                                class_mode='categorical',
#                                                subset='validation')


In [ ]:
model.fit(
    x=train_image_gen,
    epochs=7,
#    callbacks=[early_stop],
#    validation_data=test_image_gen
)

Epoch 1/7
369/369 [==============================] - 1662s 4s/step - loss: 0.6169 - precision: 0.9003
Epoch 2/7
369/369 [==============================] - 199s 539ms/step - loss: 0.2708 - precision: 0.9264
Epoch 3/7
369/369 [==============================] - 199s 540ms/step - loss: 0.2267 - precision: 0.9339
Epoch 4/7
369/369 [==============================] - 196s 532ms/step - loss: 0.1903 - precision: 0.9448
Epoch 5/7
369/369 [==============================] - 197s 532ms/step - loss: 0.1591 - precision: 0.9485
Epoch 6/7
369/369 [==============================] - 196s 532ms/step - loss: 0.1648 - precision: 0.9515
Epoch 7/7
369/369 [==============================] - 196s 531ms/step - loss: 0.1519 - precision: 0.9506


In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import glob

fname,test=[],[]

test_dir='drive/My Drive/TEST'

for infile in glob.glob(test_dir+"/*.jpg"):
  img=Image.open(infile)
  img=img.resize((400,400))
  fname.append(infile.split('/')[-1])
  img_arr=np.asarray(img)
  test.append(img_arr.reshape(400,400,3))

test_len=len(test)
test=np.array(test)

In [ ]:
maps = {v: k for k, v in train_image_gen.class_indices.items()}

In [ ]:
dense = 0
if dense == 1:
    test = test.reshape(test_len, 3072)
    
#test = test / 255.
    
y_test_pred = np.argmax(model.predict(test), axis=1).tolist()
y_test_pred = [maps[val] for val in y_test_pred]

In [ ]:
result=pd.DataFrame({'Filename':fname, 'Class':y_test_pred})
result.to_csv('res.csv',index=False)